In [ ]:
# This item imports some commonly used pre-training models
pip install bio-embeddings[all]

In [ ]:
import torch
import os
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score,confusion_matrix,roc_curve,auc,roc_auc_score
from sklearn import manifold
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split,KFold

In [ ]:
# Some tool functions
def getCls(vector):
  vector=vector.mean(axis=0)
  return vector
def data_write(input_data,output_file_name):
  embedder = ProtTransBertBFDEmbedder()
  cnt=0
  for i in input_data:
    # if cnt<645:
    #   cnt+=1
    #   continue
    embedding = embedder.embed(i)
    cls=getCls(embedding)
    cnt+=1
    print(cnt)
    print(cls)
    if not os.path.exists(output_file_name):
      os.system(r"touch {}".format(output_file_name))
    with open(output_file_name,'a') as f:
      f.write(str(cnt)+"\n")
      for j in cls:
        f.write(str(j)+" ")
      f.write("\n")
def readClsData(cls_datafile,cls_data):
    cnt=0
    with open(cls_datafile,"r") as f:
        d=f.readline()
        while d:
            d=f.readline().split() 
            if d:
                temp=[]
                for i in d:
                    i=float(i)
                    temp.append(i)
                cnt+=1
                cls_data.append(temp)
            f.readline()
def readTgData(file_name):
    lists = []
    with open(file_name, "r") as f:
        d = f.readline()
        while d != '\n' and d:
            line = f.readline()
            li = line.split(" ")
            lists.append(li)
            d = f.readline()
    return lists
def modelPerformance(y_true, y_pred):
    print('-------------------------------------------------------------')
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import roc_curve
    fpr, _, _ = roc_curve(y_pred, y_true)
    from sklearn.metrics import f1_score
    from sklearn.metrics import cohen_kappa_score
    from sklearn.metrics import matthews_corrcoef
    print('ACC: ' + str(accuracy_score(y_true, y_pred)))
    print('Precision: ' + str(precision_score(y_true, y_pred)))
    print('Sn: ' + str(recall_score(y_true, y_pred)))
    print('Sp: ' + str(1 - fpr[1]))
    print("F1: " + str(f1_score(y_true, y_pred)))
    print("Kappa: " + str(cohen_kappa_score(y_true, y_pred)))
    print("MCC: " + str(matthews_corrcoef(y_true, y_pred)))
    print('-------------------------------------------------------------')

In [ ]:
# Get origin data
path_ne_train="/Data/MyDrive/Colab Notebooks/data/negative_train.txt"
path_ne_test="/Data/MyDrive/Colab Notebooks/data/negative_test.txt"
path_po_train="/Data/MyDrive/Colab Notebooks/data/positive_train.txt"
path_po_test="/Data/MyDrive/Colab Notebooks/data/positive_test.txt"
negative_train=[]
negative_test=[]
positive_train=[]
positive_test=[]

with open (path_ne_train,"r") as f:
    d=f.readline()
    while d:
        d=f.readline().strip()
        f.readline()
        if d:
          negative_train.append(d)
    print(len(negative_train))
    
with open (path_ne_test,"r") as f:
    d=f.readline()
    while d:
        d=f.readline().strip()
        f.readline()
        if d:
            negative_test.append(d)
    print(len(negative_test))

with open (path_po_train,"r") as f:
    d=f.readline()
    while d:
        d=f.readline().strip()
        f.readline()
        if d:
            positive_train.append(d)
    print(len(positive_train))
    
with open (path_po_test,"r") as f:
    d=f.readline()
    while d:
        d=f.readline().strip()
        f.readline()
        if d:
            positive_test.append(d)
    print(len(positive_test))

In [ ]:
# Pre-training model features
path_ne_train_cls="/Data/MyDrive/Colab Notebooks/data/negative_train_cls.txt"
path_ne_test_cls="/Data/MyDrive/Colab Notebooks/data/negative_test_cls.txt"
path_po_train_cls="/Data/MyDrive/Colab Notebooks/data/positive_train_cls.txt"
path_po_test_cls="/Data/MyDrive/Colab Notebooks/data/positive_test_cls.txt"
data_write(negative_train,path_ne_train_cls)
data_write(negative_test,path_ne_test_cls)
data_write(positive_train,path_po_train_cls)
data_write(positive_test,path_po_test_cls)

In [ ]:
# This module reads pssm features, and all pssm features are generated by POSSUM. Check our paper for details.
# kp pssm
kp_ne_test = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/kp_ne_test.csv","rb"),delimiter=",",skiprows=1)
print(kp_ne_test.shape)

kp_ne_train = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/kp_ne_train.csv","rb"),delimiter=",",skiprows=1)
print(kp_ne_train.shape)

kp_po_test = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/kp_po_test.csv","rb"),delimiter=",",skiprows=1)
print(kp_po_test.shape)

kp_po_train = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/kp_po_train.csv","rb"),delimiter=",",skiprows=1)
print(kp_po_train.shape)

# dpc pssm
dpc_ne_test = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/dpc_ne_test.csv","rb"),delimiter=",",skiprows=1)
print(dpc_ne_test.shape)

dpc_ne_train = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/dpc_ne_train.csv","rb"),delimiter=",",skiprows=1)
print(dpc_ne_train.shape)

dpc_po_test = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/dpc_po_test.csv","rb"),delimiter=",",skiprows=1)
print(dpc_po_test.shape)

dpc_po_train = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/dpc_po_train.csv","rb"),delimiter=",",skiprows=1)
print(dpc_po_train.shape)

# sf pssm
sf_ne_test = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/sf_ne_test.csv","rb"),delimiter=",",skiprows=1)
print(sf_ne_test.shape)

sf_ne_train = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/sf_ne_train.csv","rb"),delimiter=",",skiprows=1)
print(sf_ne_train.shape)

sf_po_test = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/sf_po_test.csv","rb"),delimiter=",",skiprows=1)
print(sf_po_test.shape)

sf_po_train = np.loadtxt(open("/Data/MyDrive/Colab Notebooks/data/sf_po_train.csv","rb"),delimiter=",",skiprows=1)
print(sf_po_train.shape)

In [ ]:
# This module performs feature selection operations
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1,max_depth=3)
forest.fit(x_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

# combine features
a=np.hstack([np.array(positive_train_cls),kp_po_train,dpc_po_train,sf_po_train])
b=np.hstack([np.array(negative_train_cls),kp_ne_train,dpc_ne_train,sf_ne_train])
x_train=np.vstack([a,b])

a=np.hstack([np.array(positive_test_cls),kp_po_test,dpc_po_test,sf_po_test])
b=np.hstack([np.array(negative_test_cls),kp_ne_test,dpc_ne_test,sf_ne_test])
x_test=np.vstack([a,b])

y_train=np.hstack([np.ones(1005), np.zeros(1059)])
y_test=np.hstack([np.ones(218), np.zeros(260)])

x_train_select=[]
x_test_select=[]

K=1500 # This value represents the features length

for i in x_train:
  traint=[]
  for k in indices[0:K]:
    traint.append(i[k])
  x_train_select.append(traint)
for i in x_test:
  testt=[]
  for k in indices[0:K]:
    testt.append(i[k])
  x_test_select.append(testt)
x_train_select=np.array(x_train_select)
x_test_select=np.array(x_test_select)

In [ ]:
# This module performs performance testing and draws ROC curves
# There may be some redundant code written to implement different functions. Please enable the corresponding code according to the purpose, and do not run all the code at once.
# plt.figure(figsize=(15,4))
# plt.subplot(1,3,1)
# plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('T5')


x_train=x_train_select 
x_test=x_test_select 

smodel5 = svm.SVC(gamma='scale', 
          C=10, 
          decision_function_shape='ovr', 
          kernel='rbf')
smodel5.fit(x_train,y_train)
result=smodel5.predict(x_test)
modelPerformance(y_test,result)
# fpr,tpr, thresholds = roc_curve(y_test,smodel1.decision_function(x_test))
# auc = roc_auc_score(y_test,smodel1.decision_function(x_test))
# plt.plot(fpr,tpr,color='darkgoldenrod',label='SVM (AUC = %0.2f)'%auc)

forest1 = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1,max_depth=3)
forest1.fit(x_train, y_train)
result=forest1.predict(x_test)
modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,forest1.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,forest1.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='red',label='RF (AUC = %0.2f)'%auc)

NB1 = GaussianNB()
NB1.fit(x_train, y_train)
result=NB1.predict(x_test)
modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,NB1.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,NB1.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='orange',label='NB (AUC = %0.2f)'%auc)

XGB1 = XGBClassifier(max_depth=15,
          learning_rate=0.1,
          n_estimators=2000,
          min_child_weight=5,
          max_delta_step=0,
          subsample=0.8,
          colsample_bytree=0.7,
          reg_alpha=0,
          reg_lambda=0.4,
          scale_pos_weight=0.8,
          objective='binary:logistic',
          eval_metric='auc',
          seed=1440,
          gamma=0)
XGB1.fit(x_train, y_train)
y_pred = XGB1.predict(x_test)
result = [round(value) for value in y_pred]
modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,XGB1.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,XGB1.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='dodgerblue',label='XGB (AUC = %0.2f)'%auc)

knc1 = KNN(n_neighbors =5)
knc1.fit(x_train,y_train)
result = knc1.predict(x_test)
modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,knc1.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,knc1.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='purple',label='KNN (AUC = %0.2f)'%auc)
# plt.legend(loc="lower right")

plt.subplot(1,3,2)
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('PBD')


# x_train=trainPBD 
# x_test=testPBD  

# # smodel2 = svm.SVC(gamma='scale', 
# #           C=10, 
# #           decision_function_shape='ovr', 
# #           kernel='rbf')
# # smodel2.fit(x_train,y_train)
# # result=smodel2.predict(x_test)
# # modelPerformance(y_test,result)
# fpr,tpr, thresholds = roc_curve(y_test,smodel2.decision_function(x_test))
# auc = roc_auc_score(y_test,smodel2.decision_function(x_test))
# plt.plot(fpr,tpr,color='darkgoldenrod',label='SVM (AUC = %0.2f)'%auc)

# # forest2 = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1,max_depth=3)
# # forest2.fit(x_train, y_train)
# # result=forest2.predict(x_test)
# # modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,forest2.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,forest2.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='red',label='RF (AUC = %0.2f)'%auc)

# # NB2 = GaussianNB()
# # NB2.fit(x_train, y_train)
# # result=NB2.predict(x_test)
# # modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,NB2.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,NB2.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='orange',label='NB (AUC = %0.2f)'%auc)

# # XGB2 = XGBClassifier(max_depth=15,
# #           learning_rate=0.1,
# #           n_estimators=2000,
# #           min_child_weight=5,
# #           max_delta_step=0,
# #           subsample=0.8,
# #           colsample_bytree=0.7,
# #           reg_alpha=0,
# #           reg_lambda=0.4,
# #           scale_pos_weight=0.8,
# #           objective='binary:logistic',
# #           eval_metric='auc',
# #           seed=1440,
# #           gamma=0)
# # XGB2.fit(x_train, y_train)
# # y_pred = XGB2.predict(x_test)
# # result = [round(value) for value in y_pred]
# # modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,XGB2.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,XGB2.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='dodgerblue',label='XGB (AUC = %0.2f)'%auc)

# # knc2 = KNN(n_neighbors =5)
# # knc2.fit(x_train,y_train)
# # result = knc2.predict(x_test)
# # modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,knc2.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,knc2.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='purple',label='KNN (AUC = %0.2f)'%auc)
# plt.legend(loc="lower right")





# plt.subplot(1,3,3)
# plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('SeqVec')


# x_train=trainSEQ 
# x_test=testSEQ  

# # smodel3 = svm.SVC(gamma='scale', 
# #           C=10, 
# #           decision_function_shape='ovr', 
# #           kernel='rbf')
# # smodel3.fit(x_train,y_train)
# # result=smodel3.predict(x_test)
# # modelPerformance(y_test,result)
# fpr,tpr, thresholds = roc_curve(y_test,smodel3.decision_function(x_test))
# auc = roc_auc_score(y_test,smodel3.decision_function(x_test))
# plt.plot(fpr,tpr,color='darkgoldenrod',label='SVM (AUC = %0.2f)'%auc)

# # forest3 = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1,max_depth=3)
# # forest3.fit(x_train, y_train)
# # result=forest3.predict(x_test)
# # modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,forest3.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,forest3.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='red',label='RF (AUC = %0.2f)'%auc)

# # NB3 = GaussianNB()
# # NB3.fit(x_train, y_train)
# # result=NB3.predict(x_test)
# # modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,NB3.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,NB3.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='orange',label='NB (AUC = %0.2f)'%auc)

# # XGB3 = XGBClassifier(max_depth=15,
# #           learning_rate=0.1,
# #           n_estimators=2000,
# #           min_child_weight=5,
# #           max_delta_step=0,
# #           subsample=0.8,
# #           colsample_bytree=0.7,
# #           reg_alpha=0,
# #           reg_lambda=0.4,
# #           scale_pos_weight=0.8,
# #           objective='binary:logistic',
# #           eval_metric='auc',
# #           seed=1440,
# #           gamma=0)
# # XGB3.fit(x_train, y_train)
# # y_pred = XGB3.predict(x_test)
# # result = [round(value) for value in y_pred]
# # modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,XGB3.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,XGB3.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='dodgerblue',label='XGB (AUC = %0.2f)'%auc)

# # knc3 = KNN(n_neighbors =5)
# # knc3.fit(x_train,y_train)
# # result = knc3.predict(x_test)
# # modelPerformance(y_test,result)
# auc = roc_auc_score(y_test,knc3.predict_proba(x_test)[:,1])
# fpr,tpr, thresholds = roc_curve(y_test,knc3.predict_proba(x_test)[:,1])
# plt.plot(fpr,tpr,color='purple',label='KNN (AUC = %0.2f)'%auc)
# plt.legend(loc="lower right")

# # plt.subplots_adjust(left=None,bottom=None,right=None,top=None,wspace=0.3,hspace=0.3)
# plt.savefig("All Roc.svg", dpi=300,format="svg")
# plt.show()